In [1]:
from brian2 import *
%matplotlib widget
prefs.codegen.target = 'numpy'

## Efficiently creating connection patterns

In [2]:
# dummy neurons and synapses
start_scope()
neurons = NeuronGroup(200, '')
synapses = Synapses(neurons, neurons, 'w : 1')
synapses.connect()  # all-to-all

In [3]:
weight_matrix = np.load('weight_matrix.npy')

In [4]:
%%time
for source_idx in range(len(neurons)):
    for target_idx in range(len(neurons)):
        synapses.w[source_idx, target_idx] = weight_matrix[source_idx, target_idx]

CPU times: user 5.51 s, sys: 6.92 ms, total: 5.52 s
Wall time: 5.54 s


## Efficiently accessing monitored variables

In [ ]:
start_scope()

area = 20000*umetre**2
Cm = 1*ufarad*cm**-2 * area
gl = 5e-5*siemens*cm**-2 * area
El = -65*mV
EK = -90*mV
ENa = 50*mV
g_na = 100*msiemens*cm**-2 * area
g_kd = 30*msiemens*cm**-2 * area
VT = -63*mV

# The model
eqs = Equations('''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I)/Cm : volt
dm/dt = 0.32*(mV**-1)*4*mV/exprel((13.*mV-v+VT)/(4*mV))/ms*(1-m)-0.28*(mV**-1)*5*mV/exprel((v-VT-40.*mV)/(5*mV))/ms*m : 1
dn/dt = 0.032*(mV**-1)*5*mV/exprel((15.*mV-v+VT)/(5*mV))/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
I : amp
''')

group = NeuronGroup(1000, eqs,
                    threshold='v > -40*mV',
                    refractory='v > -40*mV',
                    method='exponential_euler')
group.v = El
group.I = 'rand() * 1*nA'

spike_mon = SpikeMonitor(group)
v_mon = StateMonitor(group, 'v', record=True)

run(10*second, report='text')

Starting simulation at t=0. s for a duration of 10. s


In [ ]:
%%timee
# Calculating the spike count and mean ISI (in ms) for each cell
counts = spike_mon.count
mean_ISIs = np.zeros(len(group))
spike_trains = spike_mon.spike_trains()
for neuron_idx in range(len(group)):
    spike_times = spike_trains[neuron_idx]
    mean_ISIs[neuron_idx] = np.mean(np.diff(spike_times/ms))

In [ ]:
%%timee
# Calculating the spike count and mean ISI (in ms) for each cell
counts = spike_mon.count
mean_ISIs = np.zeros(len(group))
spike_trains = spike_mon.spike_trains()
for neuron_idx in range(len(group)):
    spike_times = spike_trains[neuron_idx]
    mean_ISIs[neuron_idx] = np.mean(np.diff(spike_times/ms))